# Data Management project
*group members:*
- Giada Sansonetto, 880841@stud.unive.it, matriculation nr. 880841
- Valentina Conz, 974888@stud.unive.it, matriculation nr. 974888
- Matteo Secco, 878417@stud.unive.it, matriculation nr. 878417

# 1. Introduction

The project analyses some of the possible causes that may have worsened the damaging effects of the covid19 pandemic.

## 1.1 Datasets
The raw data has been taken from the following datasets:
- https://www.kaggle.com/tanuprabhu/population-by-country-2020?select=population_by_country_ for the data regarding countries population
- https://developers.google.com/public-data/docs/canonical/countries_csv for the data regarding latitude and longitude of countries
- https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv for the data regarding vaccinations
- https://github.com/datasets/covid-19/blob/main/data/countries-aggregated.csv for the data regarding covid infections and deaths
- https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations-by-manufacturer.csv for the data regarding vaccinations divided by manufacturer

## 1.2 Data cleaning
The raw data has subsequently been cleaned for name inconsistencies, missing/non relevant countries countries and fixed the schemas. The work has been done mainly by hand and with the help of Pyhton, resulting in the following files:
- 1_population.csv
- 2_longlat.csv
- 3_vaccinations.csv
- 4_covid.csv
- 5_manufacturers.csv

The first two files are the most extensive ones and will be merged, as they contain data for 228 countries. The others contain some missing data but they don't have any additional country/region, making the relation with the main table [1,1].

# 2. Requirement analysis and logical design

In this section we will outline the requirement analysis and its logical design implementation.

## Requirement analysis TO DO
We have a list of countries names together with their ISO, population, density, land area, med. age, urban pop, latitude and longitude. And for most (but not all) of them we have their number of covid infections/recovered/death for each day since the beginning of the pandemic. At the same time for each day we have the records for the number of people vaccinated, the number of daily vaccinations, fully vaccinated per hundred, daily vaccination per million and daily people vaccinated per hundred, and the records for the vaccine for each country for each manufacturer. We want to keep this datasets separated even if they share the minimal key as the sources of data are different and there may be discrepancies.

## Logical design
Here there's the logical design:

countries(<ins>ISO</ins>, name, population, density, land_area, med_age, lat, long)

vaccinations(<ins>date</ins>, <ins>country</ins>: fk to countries(ISO), people_fully_vac, daily_vaccinations, people_fully_vaccinated_per_hundred, daily_people_vaccinated_per_hundred)

covid(<ins>date</ins>, <ins>ISO</ins>: fk to countries(ISO), confirmed, recovered, deaths)

manufacturer(<ins>date</ins>, <ins>ISO</ins>:fk to countries(ISO), vaccine, total)

# 3. Physical design
First, we set up the libraries and the function used to load .csv files into the database.








In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sqlite3 

# deletes the file in case it is already existing
try:
    os.remove("database.db")
except OSError:
    pass

# connection with sqlite3
conn = sqlite3.connect("database.db")
cur = conn.cursor()


def run_sql(sql_command):
  # executes a SQL code
  
  cur.execute(sql_command)
  conn.commit()
  if cur.description != None:
    col_names = [i[0] for i in cur.description]
    result = pd.DataFrame(cur.fetchall(), columns = col_names)
    result
    return result


def dataframe_to_table(df,table_name):
  # converts a Pandas dataframe into a SQL table in the database.
  
  df.to_sql(table_name, conn, if_exists='replace', index = False)
  conn.commit()

We load the .csv files into python.

In [ ]:
population_df = pd.read_csv("1_population.csv")
longlat_df = pd.read_csv("2_longlat.csv")
vaccinations_df = pd.read_csv("3_vaccinations.csv")
covid_df = pd.read_csv("4_covid.csv")
manufacturers_df = pd.read_csv("5_manufacturers.csv")

Then we created the tables into the database through Python.

In [ ]:
run_sql('''
CREATE TABLE countries (
  iso CHAR(2),
  name VARCHAR(50) PRIMARY KEY,
  population INT,
  density FLOAT,
  area FLOAT,
  med_age FLOAT,
  latitude FLOAT,
  longitude FLOAT
);
''')

run_sql('''
CREATE TABLE vaccinations (
  country VARCHAR(2) references countries(name),
  date DATE,
  people_fully_vac INT,
  daily_vaccinations INT,
  people_fully_vaccinated_per_hundred INT,
  daily_people_vaccinated_per_hundred INT,
  primary key(iso, date)
);
''')

run_sql('''
CREATE TABLE covid (
  country VARCHAR(50) references countries(name),
  date DATE,
  confirmed INT,
  recovered INT,
  deaths INT,
  primary key(iso, date)
);
''')

run_sql('''
CREATE TABLE manufacturer (
  country VARCHAR(50) references countries(name),
  date DATE,
  vaccine VARCHAR(20),
  total INT,
  primary key(iso, date, vaccine)
);
''')

First, we insert the CSV files into temporary SQL tables that will be later processed to create our database.

In [ ]:
dataframe_to_table(population_df,"population_tmp")
dataframe_to_table(longlat_df,"longlat_tmp")
dataframe_to_table(vaccinations_df,"vaccinations_tmp")
dataframe_to_table(covid_df,"covid_tmp")
dataframe_to_table(manufacturers_df,"manufacturers_tmp")

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


Then we start to create the actual tables required by our database, beginning by merging 1_population.csv and 2_longlat.csv

In [ ]:
# merge 1_population.csv and 2_longlat.csv and remove useless columns
run_sql('''
INSERT INTO countries
SELECT iso, country, population, Density,"land area","med age", latitude, longitude
FROM population_tmp
INNER JOIN longlat_tmp ON population_tmp.Country=longlat_tmp.Name
''')

# load 3_vaccinations.csv and remove columns
# iso was taken from previous table
run_sql('''
insert into vaccinations
select location, "date", people_fully_vaccinated, daily_vaccinations, people_fully_vaccinated_per_hundred, daily_people_vaccinated_per_hundred
from vaccinations_tmp
''')

# load 4_covid.csv and remove columns
run_sql('''
insert into covid
select country, "date", confirmed, recovered, deaths
from covid_tmp
''')

# load 5_manufacturers.csv
run_sql('''
insert into manufacturer
select location, "date", vaccine, total_vaccinations
from manufacturers_tmp
''')

# drop all temp tables
run_sql('''
DROP TABLE population_tmp
''')
run_sql('''
DROP TABLE longlat_tmp
''')
run_sql('''
DROP TABLE vaccinations_tmp
''')
run_sql('''
DROP TABLE covid_tmp
''')
run_sql('''
DROP TABLE manufacturers_tmp
''')

# 4. Queries

We first compute the required queries
## 4.1 
For a country, consider the value VAX obtained by dividing the number of people vaccinated up to today by
the country’s population size. Return a list of all countries sorted by decreasing VAX (showing also the value
of VAX for each listed country). Careful: the numbers in the vaccination CSV are cumulative.

In [ ]:
run_sql('''
select iso, max("date"), max(people_fully_vac)
from vaccinations
group by iso
''')

,iso,"max(""date"")",people_fully_vac
0,Afghanistan,2022-05-08,4670950.0
1,Albania,2022-04-24,1235239.0
2,Algeria,2022-05-01,6481186.0
3,Andorra,2022-04-24,53414.0
4,Angola,2022-04-26,6327907.0
...,...,...,...
205,Vietnam,2022-03-22,77754108.0
206,Wallis and Futuna,2022-05-02,NaN
207,Yemen,2022-04-19,418785.0
208,Zambia,2022-05-08,NaN


#4.2
Find the country in the database that had the smallest number of confirmed covid cases per square kilometer
(up to today).

In [ ]:
run_sql('''
SELECT name, (covid.confirmed/countries.area) as 'Confirmed cases per km2'
from countries, covid
where countries.name = covid.iso
  and date = '2022-04-16'
group by covid.confirmed

''')

,name,Confirmed cases per km2
0,Micronesia,0.001429
1,Marshall Islands,0.038889
2,Vatican City,NaN
3,Kiribati,3.791358
4,Palau,9.108696
...,...,...
186,Germany,67.181154
187,France,50.906607
188,Brazil,3.619235
189,India,14.476739


#4.3
We search the country, in each empishpere of the world, that had the largest number of covid deaths relative to the population, in October 2020.

CHECK
- not working with population
- how to order by number of cases

In [ ]:
# northem emisphere

run_sql('''
SELECT name, sum(deaths)/density 
FROM countries, covid
WHERE latitude >= 0 
  and date like '2020-10-%'
  and covid.iso = countries.name
group by name

''')

,name,sum(deaths)/population
0,Afghanistan,0
1,Albania,0
2,Algeria,0
3,Andorra,0
4,Antigua and Barbuda,0
...,...,...
146,Uzbekistan,0
147,Vatican City,0
148,Venezuela,0
149,Vietnam,0


# 5. Data visualization

# 6. Discussion

### extra - to be deleted
visualization of the tables

In [ ]:
run_sql('''
select *
from covid
''')

,iso,date,confirmed,recovered,deaths
0,Afghanistan,2020-01-22,0,0,0
1,Afghanistan,2020-01-23,0,0,0
2,Afghanistan,2020-01-24,0,0,0
3,Afghanistan,2020-01-25,0,0,0
4,Afghanistan,2020-01-26,0,0,0
...,...,...,...,...,...
155851,Zimbabwe,2022-04-12,247094,0,5460
155852,Zimbabwe,2022-04-13,247160,0,5460
155853,Zimbabwe,2022-04-14,247208,0,5462
155854,Zimbabwe,2022-04-15,247237,0,5462


In [ ]:
run_sql('''
select *
from countries
''')

,iso,name,population,density,area,med_age,latitude,longitude
0,CN,China,1440297825,153.0,9388211.0,38.0,35.861660,104.195397
1,IN,India,1382345085,464.0,2973190.0,28.0,20.593684,78.962880
2,US,United States,331341050,36.0,9147420.0,38.0,37.090240,-95.712891
3,ID,Indonesia,274021604,151.0,1811570.0,30.0,-0.789275,113.921327
4,PK,Pakistan,221612785,287.0,770880.0,23.0,30.375321,69.345116
...,...,...,...,...,...,...,...,...
223,MS,Montserrat,4993,50.0,100.0,N.A.,16.742498,-62.187366
224,FK,Falkland Islands,3497,0.0,12170.0,N.A.,-51.796253,-59.523613
225,NU,Niue,1628,6.0,260.0,N.A.,-19.054445,-169.867233
226,TK,Tokelau,1360,136.0,10.0,N.A.,-8.967363,-171.855881


In [ ]:
run_sql('''
select *
from vaccinations
''')

,iso,date,people_fully_vac,daily_vaccinations,people_fully_vaccinated_per_hundred,daily_people_vaccinated_per_hundred
0,Afghanistan,2021-02-22,NaN,NaN,NaN,NaN
1,Afghanistan,2021-02-23,NaN,1367.0,NaN,0.003
2,Afghanistan,2021-02-24,NaN,1367.0,NaN,0.003
3,Afghanistan,2021-02-25,NaN,1367.0,NaN,0.003
4,Afghanistan,2021-02-26,NaN,1367.0,NaN,0.003
...,...,...,...,...,...,...
89948,Zimbabwe,2022-05-04,3714905.0,17194.0,24.61,0.030
89949,Zimbabwe,2022-05-05,NaN,21512.0,NaN,0.029
89950,Zimbabwe,2022-05-06,3777867.0,25407.0,25.03,0.031
89951,Zimbabwe,2022-05-07,3827660.0,33709.0,25.36,0.041


In [ ]:
run_sql('''
select *
from manufacturer
where iso = 'Argentina' 
''')

NameError: ignored

In [ ]:
run_sql('''
select covid.deaths
from covid
where date like '2022-04-16'
  

''')

In [ ]:
pollution_df = pd.read_csv('worldairquality.csv')

ParserError: ignored

### Query on pollution
We question whether there exists a relation between air pollution and covid cases.

In [ ]:
pollution_df = pd.read_csv('worldairquality.csv')

run_sql('''
CREATE TABLE pollution
  iso char(2)
  country varchar(30)
  value int
  primary key(iso, value)
''')

dataframe_to_table(pollution_df, 'pollution_tmp')

run_sql('''
INSERT INTO pollution
SELECT iso, country, value
FROM pollution_tmp

''')

# query – is there a relation between pollution and covid cases?
# firstly, we produce the data that we need

run_sql('''
select sum(value) 
from pollution
where 

''')



ParserError: ignored

### Query
We take a look at the first countries in which covid manifested.

In [ ]:

run_sql('''

select 
from 
where 
group by date

''')
